#### For Colab

In [1]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [2]:
!nvidia-smi

Wed Mar 17 14:33:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [5]:
!cd '/content/drive/My Drive/MAGMA: Summarization/transformers/'; pip install -q -e .
!pip install -q -U sentence-transformers
!pip install -q -U wandb
!pip install -q -U rouge-score sacrebleu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 22.6MB/s 
     |████████████████████████████████| 890kB 39.6MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 1.2MB 23.3MB/s 
     |████████████████████████████████| 2.0MB 16.4MB/s 
     |████████████████████████████████| 163kB 56.5MB/s 
     |████████████████████████████████| 133kB 55.9MB/s 
     |████████████████████████████████| 102kB 15.7MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 


In [6]:
print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

Setting up colab environment
     |████████████████████████████████| 47.5MB 66kB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 204kB 61.7MB/s 
     |████████████████████████████████| 1.3MB 45.6MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 133kB 51.1MB/s 
     |████████████████████████████████| 3.2MB 53.8MB/s 
     |████████████████████████████████| 317kB 56.7MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 296kB 57.7MB/s 
     |████████████████████████████████| 143kB 58.4MB/s 


In [7]:
import sys

print("\n##### sys.path original #####")
for p in sys.path:
    print(f"'{p}'")
print()

import site
site.main()

print("\n##### sys.path after site.main() #####")
for p in sys.path:
    print(f"'{p}'")
print()


##### sys.path original #####
''
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'


##### sys.path after site.main() #####
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'
'/content/drive/My Drive/MAGMA: Summarization/transformers/src'



### **Config**

In [8]:
import sys
sys.path.insert(0, drive_dir)
import config

import wandb
wandb.login()

project_name = 'ft_pegasus_bull_para_embed'
%env WANDB_PROJECT=$project_name

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=ft_pegasus_para_wordembed


In [9]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'
gps_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval_search.py"'

In [10]:
model_name_or_path = 'google/pegasus-large'

In [11]:
data_dir = '"'+drive_dir+'datasets/bullet_paragraph_embeddings/pegasus/st/base/"'

source_val_dir = data_dir[:-1] + 'val.source"'
reference_val_dir = data_dir[:-1] + 'val.target"'

source_test_dir = data_dir[:-1] + 'test.source"'
reference_test_dir = data_dir[:-1] + 'test.target"'

In [12]:
GAS = 64
LR = 5e-5

output_dir = '"'+drive_dir+'fine-tuning/'+project_name+'_gas'+\
    str(GAS)+'_lr' + str(LR) + '"'

### Evaluation Out of the Box

#### Validation

In [ ]:
eval_model_name_or_path = '"google/pegasus-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_val.json"'
print(save_dir)

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_gas64_lr5e-05/google?pegasus-large_para_wordembed_val.txt"
100% 306M/306M [00:35<00:00, 8.63MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
Downloading: 100% 2.87k/2.87k [00:00<00:00, 4.43MB/s]
Downloading: 100% 2.28G/2.28G [00:41<00:00, 54.3MB/s]
Downloading: 100% 1.91M/1.91M [00:00<00:00, 29.5MB/s]
Downloading: 100% 65.0/65.0 [00:00<00:00, 113kB/s]
Downloading: 100% 88.0/88.0 [00:00<00:00, 148kB/s]
  0% 0/34 [00:00<?, ?it/s]2021-03-07 10:52:30.901500: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [05:02<00:00,  8.90s/it]
{'rouge1_precision': 25.230000000000004, 'rouge1_recall': 34.01, 'rouge1_fmeasure': 27.43, 'rouge2_precision': 8.01, 'rouge2_recall': 11.09, 'rouge2_fmeasure': 8.780000000000001, 'rougeL_precision': 18.77, 'rougeL_recall': 25.580000000000002, 'rougeL_fmeasure': 20.46, '

#### Test

In [ ]:
eval_model_name_or_path = '"google/pegasus-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/31 [00:00<?, ?it/s]2021-03-07 11:08:57.021355: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 31/31 [04:26<00:00,  8.60s/it]
{'rouge1_precision': 21.44, 'rouge1_recall': 32.74, 'rouge1_fmeasure': 24.6, 'rouge2_precision': 7.2700000000000005, 'rouge2_recall': 11.360000000000001, 'rouge2_fmeasure': 8.42, 'rougeL_precision': 16.7, 'rougeL_recall': 25.979999999999997, 'rougeL_fmeasure': 19.3, 'rougeLsum_precision': 17.2, 'rougeLsum_recall': 26.44, 'rougeLsum_fmeasure': 19.75, 'sentence_distilroberta_cosine': 53.72946858406067, 'w2v_cosine': 99.99577403068542, 'n_obs': 242, 'runtime': 266, 'seconds_per_sample': 1.0992, 'min_length': 10, 'max_length': 150}


### Fine-tuning

In [ ]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config_dir = '"'+drive_dir+'fine-tuning/config/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--freeze_embeds \
--adafactor \
--do_train \
--learning_rate $LR \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--max_steps 700 \
--logging_steps 7 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps $GAS \
--evaluation_strategy steps --eval_steps 7 --eval_beams 2 \
--predict_with_generate \
--save_steps 49 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

Streaming output truncated to the last 5000 lines.


Batches: 100%|██████████| 1/1 [00:00<00:00, 134.49it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 140.01it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 136.38it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 145.58it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 134.58it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 146.66it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 134.27it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.42it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 117.23it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 108.36it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 118.10it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 124.45it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 122.51it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.90it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 139.73it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 137.76it/s]


Batches: 100%|█████

### GPS

In [ ]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-686"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_gps.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_gps.json"'

!python3 $gps_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--task summarization \
--search="min_length=5:10:15 length_penalty=0.1:1:1.9 num_beams=4:6 no_repeat_ngram_size=3:4:5" \
--dump-args \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_gas64_lr5e-05/checkpoint-686"

  0% 0/34 [00:00<?, ?it/s]2021-03-05 10:35:50.972083: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [01:52<00:00,  3.31s/it]
100% 34/34 [01:46<00:00,  3.13s/it]
100% 34/34 [01:46<00:00,  3.13s/it]
100% 34/34 [02:17<00:00,  4.03s/it]
100% 34/34 [02:16<00:00,  4.02s/it]
100% 34/34 [02:16<00:00,  4.02s/it]
100% 34/34 [02:11<00:00,  3.87s/it]
100% 34/34 [02:13<00:00,  3.93s/it]
100% 34/34 [02:08<00:00,  3.78s/it]
100% 34/34 [02:53<00:00,  5.11s/it]
100% 34/34 [03:02<00:00,  5.36s/it]
100% 34/34 [03:01<00:00,  5.32s/it]
100% 34/34 [02:52<00:00,  5.07s/it]
100% 34/34 [03:01<00:00,  5.33s/it]
100% 34/34 [03:02<00:00,  5.38s/it]
100% 34/34 [03:52<00:00,  6.85s/it]
100% 34/34 [04:03<00:00,  7.16s/it]
100% 34/34 [04:12<00:00,  7.42s/it]
100% 34/34 [01:45<00:00,  3.11s/it]
100% 34/34 [01:45<00:00,  3.1

### Evaluation

In [13]:
### Parameters found through Generative Parameter Search and visual representation
### check the file gps_pegasus_para_wordembed

min_length = 10
length_penalty = 1.9
num_beams = 4
no_repeat_ngram_size = 4

In [15]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-686"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $min_length \
--length_penalty $length_penalty \
--num_beams $num_beams \
--no_repeat_ngram_size $no_repeat_ngram_size \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_gas64_lr5e-05/checkpoint-686"

parsed the following generate kwargs: {'min_length': 10, 'length_penalty': 1.9, 'num_beams': 4, 'no_repeat_ngram_size': 4, 'max_length': 150}
  0% 0/34 [00:00<?, ?it/s]2021-03-17 14:41:27.039083: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [02:26<00:00,  4.32s/it]
{'rouge1_precision': 27.35, 'rouge1_recall': 46.04, 'rouge1_fmeasure': 32.18, 'rouge2_precision': 12.559999999999999, 'rouge2_recall': 21.5, 'rouge2_fmeasure': 14.84, 'rougeL_precision': 21.36, 'rougeL_recall': 36.1, 'rougeL_fmeasure': 25.14, 'rougeLsum_precision': 22.770000000000003, 'rougeLsum_recall': 38.83, 'rougeLsum_fmeasure': 26.900000000000002, 'sentence_distilroberta_cosine': 62.72503137588501, 'w2v_cosine': 99.99638795852661, 'n_obs': 267, 'runtime': 146, 'seconds_per_sample': 0.5468, 'min_length': 10, 'length_penalty':

In [16]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-686"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+'_bull_para_embed_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $min_length \
--length_penalty $length_penalty \
--num_beams $num_beams \
--no_repeat_ngram_size $no_repeat_ngram_size \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_pegasus_para_wordembed_gas64_lr5e-05/checkpoint-686"

parsed the following generate kwargs: {'min_length': 10, 'length_penalty': 1.9, 'num_beams': 4, 'no_repeat_ngram_size': 4, 'max_length': 150}
  0% 0/31 [00:00<?, ?it/s]2021-03-17 14:45:50.484765: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 31/31 [02:09<00:00,  4.19s/it]
{'rouge1_precision': 22.6, 'rouge1_recall': 44.07, 'rouge1_fmeasure': 28.060000000000002, 'rouge2_precision': 8.870000000000001, 'rouge2_recall': 18.22, 'rouge2_fmeasure': 11.16, 'rougeL_precision': 17.75, 'rougeL_recall': 34.62, 'rougeL_fmeasure': 21.97, 'rougeLsum_precision': 18.84, 'rougeLsum_recall': 37.11, 'rougeLsum_fmeasure': 23.41, 'sentence_distilroberta_cosine': 60.97248196601868, 'w2v_cosine': 99.99634027481079, 'n_obs': 242, 'runtime': 129, 'seconds_per_sample': 0.5331, 'min_length': 10, 'length_penalty': 1.9, 'num_be